# Best model performance analysis - fasttext

This file is of more exploratory kind. It checks what kind of posts affect the missclassification of the model. In order to use it you need a pretrained model and a data set that would match chosen model. Unfortunately, in our case it is challenging to classify any patterns of speach that would affect the performence, because o the applied text preprocessing methods.  

In [ ]:
# imports
import os
import fasttext
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
# loading models

model_path_14 = r"fasttext_models\model_raw\model14.bin"
model_path_11 = r"fasttext_models\model_raw\model11.bin"
model_14 =  fasttext.load_model(model_path_14)
model_11 = fasttext.load_model(model_path_11)

In [11]:
# loading data sets

data_path_14 = r'data_preprocessed_df\\gender_df_preprocessed_14.json'
df_14 = pd.read_json(data_path_14)

data_path_11 = r'data_preprocessed_df\\gender_df_preprocessed_11.json'
df_11 = pd.read_json(data_path_11)

In [12]:
seed_value = 0
# getting the data frames with target and predictors
X_14 = df_14[df_14.columns[0]]
y_14 = df_14[df_14.columns[1]]
# splitting of the data for model14
X_train_14, X_test_14, y_train_14, y_test_14 = train_test_split(X_14, y_14, test_size=0.2, random_state=seed_value)

# getting the data frames with target and predictors
X_11 = df_11[df_11.columns[0]]
y_11 = df_11[df_11.columns[1]]
# splitting of the data for model 11
X_train_11, X_test_11, y_train_11, y_test_11 = train_test_split(X_11, y_11, test_size=0.2, random_state=seed_value)

In [40]:
y_pedicted_14 = [int(model_14.predict(text)[0][0][-1]) for text in X_test_14]
y_predicted_11 = [int(model_11.predict(text)[0][0][-1]) for text in X_test_11]

# connect pd.series to data frames
df_test_14 = pd.concat([X_test_14, y_test_14], axis= 1)
df_test_11 = pd.concat([X_test_11, y_test_11], axis= 1)

df_test_14["predicted"] = y_pedicted_14
df_test_11["predicted"] = y_predicted_11

In [41]:
# gets the confusion matrix for model 14
conf_matrix_14 = confusion_matrix(df_test_14['female'], df_test_14["predicted"])
conf_matrix_14,df_test_14['female'].value_counts()

(array([[4268,  467],
        [ 547, 3645]], dtype=int64),
 female
 0    4735
 1    4192
 Name: count, dtype: int64)

In [42]:
# rate of classification for man and women for model 14, format: (man, woman)

4268/4735, 3645/4192

(0.9013727560718057, 0.8695133587786259)

In [43]:
# gets the confusion matrix for model 11
conf_matrix_11 = confusion_matrix(df_test_11['female'], df_test_11["predicted"])
conf_matrix_11, df_test_11['female'].value_counts()

(array([[4274,  461],
        [ 559, 3633]], dtype=int64),
 female
 0    4735
 1    4192
 Name: count, dtype: int64)

In [44]:
# rate of classification for man and women for model 11, format: (man, woman)
4274/4735, 3633/4192

(0.9026399155227033, 0.8666507633587787)

In [88]:
# getting the probability of chosen label per post, there was an idea to check posts where the model missclasifies the most
# this code work well with quick model like fasttext, it is not reccomneded to make such codes for more reasource consuming models
df_test_11["probability"] = df_test_11['post'].apply(lambda x: model_11.predict(x)[1][0])
df_test_14["probability"] = df_test_14['post'].apply(lambda x: model_14.predict(x)[1][0])

In [93]:
# creating copy of main data frames, to see the distribution of given missclasifications

x = df_test_11[df_test_11["female"] != df_test_11["predicted"]].sort_values(by='probability', ascending=False)
x = x[(x['probability'] >= 0.90) & (x['probability'] <= 1)]
x[['female']].value_counts()

female
1         80
0         59
Name: count, dtype: int64

In [87]:
# saving given data frame in order to get te
x.to_csv("distribution_of_erros_model_11_range_0.9-1.csv")